In [1]:
import os
from tqdm import tqdm
import argparse

import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("sample_train.ipynb"))))
from common.parser import yaml_parser
from common.recoder import save_checkpoint
from common.cocoeval4yolo import CocoEval
from data.yolo_dataset import *
# from model.loss import YOLOv3Loss
from model.model import Darknet4YOLOv3
from model.darknet2pytorch import DarknetParser, yolo_forward_dynamic

import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter



In [4]:
import easydict

args = easydict.EasyDict({
    "config": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/model/yolov3.cfg",
    # "config": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/darknet/yolov4.cfg",
    "weight": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/darknet/yolov3-608.weights",
    "dataset": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/dataset/yolo_dataset.yml",
    "model": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/model/yolo_model.yml",
    "optimizer": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/optimizer/optimizer.yml",
    "weight_save_dir": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/weights"
})


dataset_option = yaml_parser(args.dataset)
model_option = yaml_parser(args.model)
optimizer_option = yaml_parser(args.optimizer)

In [5]:
######################
## BUILD DATALOADER ##
######################
# train_set_num, train_loader, _ = build_DataLoader(dataset_option, model_option, optimizer_option)

train_dataset = YoloDataset(dataset_option, model_option, split="train")
valid_dataset = YoloDataset(dataset_option, model_option, split="valid")
train_loader = DataLoader(train_dataset,
                          batch_size=optimizer_option["OPTIMIZER"]["BATCH_SIZE"],
                          shuffle=True,
                          collate_fn=collate_fn,
                          drop_last=True)
valid_loader = DataLoader(valid_dataset,
                          batch_size=optimizer_option["OPTIMIZER"]["BATCH_SIZE"],
                          shuffle=True,
                          collate_fn=collate_fn,
                          drop_last=True)

In [6]:
# device = torch.device('cpu')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

###########################
## BUILD MODEL & LOSS_fn ##
###########################
# model = DarknetParser(args.config, args.weight).to(device)
model = Darknet4YOLOv3(args.config, args.weight).to(device)
model = torch.nn.DataParallel(model)
# loss_function = yolo_forward_dynamic
# loss_function = YOLOv3Loss()

load weights from : 'C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/darknet/yolov3-608.weights'
Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
0 convolutional load weights : [0.004]/[236.518] mb
Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
1 convolutional load weights : [0.075]/[236.518] mb
Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
2 convolutional load weights : [0.083]/[236.518] mb
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
3 convolutional load weights : [0.155]/[236.518] mb
4 shortcut     load weights : [0.155]/[236.518] mb
Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
5 convolutional load weights : [0.438]/[236.518] mb
Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
6 convolutional load weights : [0.470]/[236.518] mb
Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
7 convolutional load weights :

In [7]:
model.eval()

coco_evaluator = CocoEval()

with torch.no_grad():
    for i, batch_input in enumerate(tqdm(valid_loader, desc="valid")):
        batch_img = batch_input["img"].to(device)

        preds = model(batch_img)                             ## preds = [(boxes, confs), (boxes, confs), (boxes, confs)]
        
        
        coco_evaluator.update(batch_input, preds)
    
mAP = coco_evaluator.eval()
print(f"mAP: {mAP}")

valid: 100%|██████████| 250/250 [02:57<00:00,  1.41it/s]


creating index...
index created!
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.55s).
Accumulating evaluation results...
DONE (t=0.07s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.164
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.566
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.016
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.109
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.198
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.202
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.072
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.227
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.227
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.185
 Average Recall     (